Gabor ML - Problem 16

In [26]:
import pandas as pd
import numpy as np
import random
import time
import matplotlib.pyplot as plt

In [27]:
#create sample data based on HW problem
def create_sample_data(n, d):

    #covariance matrix
    cov = np.identity(d)    
    for i in range(d):
        cov[i,i] = 1/(i+1)

    #x sample data
    x = np.random.multivariate_normal(np.zeros(d), cov, n)

    #y sample data
    y = np.zeros(n)
    e = np.random.normal(0, 1, n)
    for i in range(n):
        y[i] = sum(x[i]) + e[i]
    return x, y

#OLS algorithm
def run_algorithm_OLS(x, y, n, d):

    #OLS
    hold_matrix = np.zeros((d,d))
    for i in range(n):
        maxtrix_add = np.outer(x[i], x[i].T)
        hold_matrix = np.add(hold_matrix, maxtrix_add)
    hold_matrix = np.multiply(1/n, hold_matrix)
    hold_matrix = np.linalg.inv(hold_matrix)

    hold_vec = np.zeros(d)
    for i in range(n):
        add_vec = np.multiply(x[i], y[i])
        hold_vec = np.add(hold_vec, add_vec)
    hold_vec = np.multiply(1/n, hold_vec)

    w = np.dot(hold_matrix, hold_vec)
    return w

#function to create a learning rate list
def learning_rate(method_learning_rate, n):
    #create a list of what the learning rate will be at step t
    if method_learning_rate == "static":
        learning_rate_list = [.1 for i in range(n)]
    else:
        learning_rate_list = [1/np.sqrt(i+1) for i in range(n)]
    return learning_rate_list

#function that calculates the average of parameters
def average_w(w, n, d):
    #calculate average of parameters
    avg_w = np.zeros(d)
    for t in range(n):
        avg_w = np.add(avg_w, w[t])
    avg_w = np.multiply(1/n, avg_w)
    return avg_w

#Gradient Descent
def run_algorithm_GD(x, y, method_learning_rate, n, d):

    #create a learning rate list based on method
    learning_rate_list = learning_rate(method_learning_rate, n)

    #create a matrix of parameters at each step
    w = np.zeros((n,d))
    #with an initial vector
    w0 = np.zeros(d)

    #follow details from homework
    for t in range(n):
        hold_vector_add = np.zeros(d)
        if t == 0:
            for i in range(n):
                hold1 = np.multiply(np.dot(w0.T,x[i]),x[i])
                hold2 = np.multiply(x[i], y[i])
                hold3 = np.subtract(hold1, hold2)
                hold_vector_add = np.add(hold_vector_add, hold3)
            hold_vector_add = np.multiply(learning_rate_list[t],hold_vector_add)
            hold_vector_add = np.multiply(2/n,hold_vector_add)
            w[t] = np.subtract(w0, hold_vector_add)
        else:
            for i in range(n):
                hold1 = np.multiply(np.dot(w[t-1].T,x[i]),x[i])
                hold2 = np.multiply(x[i], y[i])
                hold3 = np.subtract(hold1, hold2)
                hold_vector_add = np.add(hold_vector_add, hold3)
            hold_vector_add = np.multiply(learning_rate_list[t],hold_vector_add)
            hold_vector_add = np.multiply(2/n,hold_vector_add)
            w[t] = np.subtract(w[t-1], hold_vector_add)

    avg_w = average_w(w, n, d)
    return avg_w

#Stochastic Gradient Descent
def run_algorithm_SGD(x, y, method_learning_rate, n, d):

    #create a learning rate list based on method
    learning_rate_list = learning_rate(method_learning_rate)
    
    w = np.zeros((n,d))
    w0 = np.zeros(d)

    #follow details from homework
    for t in range(n):
        if t == 0:
            hold1 = np.multiply(np.dot(w0.T,x[t]),x[t])
            hold2 = np.multiply(x[t], y[t])
            hold_vec = np.subtract(hold1, hold2)
            hold_vec = np.multiply(2,hold_vec)
            hold_vec = np.multiply(learning_rate_list[t],hold_vec)
            w[t] = np.subtract(w0, hold_vec)
        else:
            hold_vec = np.subtract(np.multiply(np.dot(w[t-1].T,x[t]),x[t]), np.multiply(x[t], y[t]))
            hold_vec = np.multiply(2,hold_vec)
            hold_vec = np.multiply(learning_rate_list[t],hold_vec)
            w[t] = np.subtract(w[t-1], hold_vec)

    avg_w = average_w(w, n, d)
    return avg_w

#calculate mean square error
def mse_calc(x, y, w, n):
    mse = 0
    for i in range(n):
        mse += np.square(np.dot(w.T,x[i])-y[i])
    mse = mse/n
    return mse

#put the way I run everything into a function
def calc_method_runs(num_iterations, iter_num, n_values, d_values, type_method, method_learning_rate):
    track_results = pd.DataFrame()
    while iter_num < num_iterations:
        print(iter_num)
        for n in n_values:
            for d in d_values:
                train_x, train_y = create_sample_data(n, d)
                test_x, test_y = create_sample_data(n, d)
                for method in type_method:
                    start = time.time()
                    if method == 'OLS':
                        w = run_algorithm_OLS(train_x, train_y)
                    elif method == 'GD':
                        w = run_algorithm_GD(train_x, train_y, method_learning_rate)
                    else:
                        w = run_algorithm_SGD(train_x, train_y, method_learning_rate)
                    end = time.time()
                    train_mse = mse_calc(train_x, train_y, w)
                    test_mse = mse_calc(test_x, test_y, w)
                    run_time = (end - start)
                    line_ = pd.DataFrame([[method, n, d, run_time, train_mse, test_mse]], columns = ['method','n_value','d_value','run_time','train_mse','test_mse'])
                    track_results = pd.concat([track_results, line_], ignore_index = True)
                    track_results.reset_index(drop = True, inplace = True)
        iter_num += 1
    return track_results


In [ ]:
#parameters for functions
type_method = ['OLS','GD','SGD']
n_values = [25, 50, 100, 250, 500]
d_values = [5, 10, 25, 50, 100, 150]

#average over the below iterations
num_iterations = 10
iter_num = 0

In [28]:
#performance when learning rate is static.
method_learning_rate = "static"
static_run = calc_method_runs(num_iterations, iter_num, n_values, d_values, type_method, method_learning_rate)

0
1
2
3
4


In [ ]:
#performance when learning rate is dynamic.
method_learning_rate = "dynamic"
dynamic_run = calc_method_runs(num_iterations, iter_num, n_values, d_values, type_method, method_learning_rate)